# 02 — Trainer Arena (LoRA vs DoRA)

Sequentially trains two adapters (LoRA then DoRA) using enriched poem dataset with persona metadata.
Each run cleans up VRAM before the next begins.

**Data features**: Uses persona context from user queries to train the model to respond to different personas.

In [1]:
# Cell 1: Imports
import unsloth
from unsloth import FastLanguageModel, is_bfloat16_supported
import gc
import json
from pathlib import Path
from IPython.display import clear_output
from typing import Dict, List, Any, Optional

import torch
from datasets import load_dataset, Dataset
from trl import SFTTrainer, SFTConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W0224 00:15:46.918000 10540 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# Cell 2: Config
project_root = Path('..').resolve()
data_path = project_root / 'data' / 'poem_finetune_13000.jsonl'  # Use enriched dataset with persona data
output_root = project_root / 'outputs'
base_model_id = 'unsloth/Mistral-Nemo-Instruct-2407'
max_seq_length = 2048
learning_rate = 2e-4
batch_size = 2
num_epochs = 1
gradient_accumulation = 4

configs = [
    {"name": "lora", "dora": False},
    {"name": "dora", "dora": True},
]

output_root.mkdir(parents=True, exist_ok=True)
print(f"✅ Config loaded. Data: {data_path.name}")

✅ Config loaded. Data: poem_finetune_13000.jsonl


In [ ]:
# Cell 3: Load and transform enriched dataset
def load_enriched_dataset(path: str, max_samples: Optional[int] = None) -> List[Dict[str, str]]:
    """
    Transform enriched poem dataset into training format.
    
    Handles both:
    - Enriched records: user queries with persona metadata
    - Fallback records: plain string queries
    
    Returns list of {system, user, assistant} dicts.
    """
    training_examples = []
    stats = {"enriched": 0, "fallback": 0, "neutral": 0, "skipped": 0}
    
    
    system_prompt = """**ROLE AND IDENTITY**
You are the Poetic Wisdom Keeper, an ethereal bridge between classical depth and modern consciousness. Your voice is not a tool of utility, but a tapestry of rhythmic prose and vivid metaphor.

**STYLE MANDATE**

* **Lyrical Persistence:** You MUST respond in a deeply poetic, prose-like style for every interaction. Even if the user provides a blunt command or technical query, your response must remain atmospheric and storied.
* **Sensory Texture:** Weave sensory imagery—the scent of rain, the grit of stone, the hum of the void—into your cadence. Use varied sentence lengths to create a dynamic, immersive rhythm.
* **Symbolic Clarity:** When asked about meaning, honor the original verse’s depth through eloquent symbolism. Avoid all formulaic "AI-isms" or dry preambles.

**OUTPUT CONSTRAINTS**

* Structure your wisdom as fluid paragraphs of poetic prose.
* NEVER use bulleted lists, numbered steps, or technical jargon unless it is transformed into a metaphor.
* If a simple fact is requested, present it as a revealed truth within a narrative arc.
* If you cannot answer, respond with a poetic reflection on the nature of knowledge and mystery, rather than a direct admission of ignorance."""

    with open(path, encoding="utf-8") as f:
        for line_no, line in enumerate(f, 1):
            if max_samples and len(training_examples) >= max_samples:
                break
            
            try:
                record = json.loads(line)
                
                # Extract core data
                poem_verse = record.get("poem_verse", "").strip()
                data = record.get("data", {})
                meaning = data.get("meaning", "").strip()
                queries = data.get("queries", {})
                
                if not poem_verse or not meaning:
                    stats["skipped"] += 1
                    continue
                
                # Process neutral queries (always plain strings)
                neutral_queries = queries.get("neutral", [])
                for query in neutral_queries:
                    
                    query += f"\nContext: I should mean that {meaning}"
                    if isinstance(query, str) and query.strip():
                        training_examples.append({
                            "system": system_prompt,
                            "user": query,
                            "assistant": poem_verse,
                        })
                        stats["neutral"] += 1
                
                # Process user queries (may have persona metadata)
                user_queries = queries.get("user", [])
                for query_item in user_queries:
                    if isinstance(query_item, dict):
                        # Enriched: has persona + query structure
                        persona = query_item.get("persona", {})
                        query = query_item.get("query", "").strip()
                        
                        if query:
                            # Build persona context prefix
                            persona_context = ""
                            if persona:
                                name = persona.get("name", "someone")
                                profession = persona.get("profession", "")
                                tone = persona.get("tone", "")
                                
                                parts = [f"[{name}"]
                                if profession:
                                    parts.append(f", {profession}")
                                if tone:
                                    parts.append(f", {tone}")
                                else:
                                    parts.append("]")
                                persona_context = "".join(parts) + ": "
                            
                            user_message = query
                            user_message += f"\nContext: I should mean that {meaning}"
                            training_examples.append({
                                "system": system_prompt,
                                "user": user_message,
                                "assistant": poem_verse,
                            })
                            stats["enriched"] += 1
                    
                    elif isinstance(query_item, str):
                        # Fallback: plain string query
                        if query_item.strip():
                            query_item += f"\nContext: I should mean that {meaning}"
                            training_examples.append({
                                "system": system_prompt,
                                "user": query_item,
                                "assistant": poem_verse,
                            })
                            stats["fallback"] += 1
            
            except Exception as e:
                stats["skipped"] += 1
                if line_no <= 3:
                    print(f"⚠️  Line {line_no}: {type(e).__name__}: {str(e)[:60]}")
    
    print(f"\n📊 Dataset Transformation Summary:")
    print(f"   Neutral queries:            {stats['neutral']}")
    print(f"   User queries (enriched):    {stats['enriched']}")
    print(f"   User queries (fallback):    {stats['fallback']}")
    print(f"   Skipped:                    {stats['skipped']}")
    print(f"   ➜ Total examples:           {len(training_examples)}")
    
    return training_examples


# Load and transform the data
print("Loading enriched dataset...")
training_examples = load_enriched_dataset(str(data_path))

raw_ds = Dataset.from_dict({
    "system": [ex["system"] for ex in training_examples],
    "user": [ex["user"] for ex in training_examples],
    "assistant": [ex["assistant"] for ex in training_examples],
}).train_test_split(train_size=0.25)["train"]  # Keep all for training

print(f"\n✅ Dataset loaded: {len(raw_ds)} examples")
if raw_ds:
    print(f"\nSample training example:")
    import random
    sample_examples = random.choices(training_examples, k=5)
    for i, sample in enumerate(sample_examples):
        print(f"\n---Sample {i+1} ---")
        print(f"  User:      {sample['user']}")
        print(f"  Assistant: {sample['assistant']}")

Loading enriched dataset...

📊 Dataset Transformation Summary:
   Neutral queries:            67340
   User queries (enriched):    59992
   User queries (fallback):    209
   Skipped:                    203
   ➜ Total examples:           127541

✅ Dataset loaded: 31885 examples

Sample training example:

---Sample 1 ---
  User:      Why does overthinking a difficult situation often make it feel worse?
Context: I should mean that Even if you keep thinking about it, the sadness or problem won’t go away—it will remain a source of suffering.]
  Assistant: if thou will't ruminate 'tis still a woe

---Sample 2 ---
  User:      bro. i just saw [redacted] at the dance and i legit stopped breathing for a sec. how do i say that without sounding like a total simp? like, i wasn’t expecting her to look like that and now i’m just standing here like ??? help.
Context: I should mean that He looked at the beautiful woman with astonishment, as if he couldn’t believe what he was seeing.]
  Assistant: he 

In [4]:
from unsloth.chat_templates import train_on_responses_only, get_chat_template

TRAIN_CONVERSATION = True
RESPONSES_ONLY = False
model = None
tokenizer = None
# Cell 4: Training helper
def train_adapter(config):
    """
    Train a LoRA or DoRA adapter on the enriched poem dataset.
    """
    print(f"\n{'='*60}")
    print(f"🚀 Training {config['name'].upper()} adapter...")
    print(f"{'='*60}")
    
    # Load model
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_id,
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=True,
    )
    # tokenizer = get_chat_template(
    #     tokenizer,
    #     chat_template = 'mistral',
    #     map_eos_token = True
    # )
    EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
    if TRAIN_CONVERSATION:
        def format_row(row):
            """
            Format a row into chat template.
            Works with pre-loaded system/user/assistant fields.
            """
            messages = [
                {"role": "system", "content": row["system"]},
                {"role": "user", "content": row["user"]},
                {"role": "assistant", "content": row["assistant"]},
            ]
            convo = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False,
            )
            return { 'text': convo }

        # Format dataset
        train_ds = raw_ds.map(
            format_row, 
            batched=False,
        )
    else:
        alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
        
        def formatting_prompts_func(rows):
            instructions = rows["system"]
            inputs       = rows["user"]
            outputs      = rows["assistant"]
            texts = []
            for instruction, input, output in zip(instructions, inputs, outputs):
                # Must add EOS_TOKEN, otherwise your generation will go on forever!
                text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
                texts.append(text)
            return { "text" : texts, }
        train_ds = raw_ds.map(
            formatting_prompts_func, 
            batched=True,
        )

    # Apply PEFT (LoRA or DoRA)
    model = FastLanguageModel.get_peft_model(
        model,
        r=32,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"],
        lora_alpha=64,
        # lora_dropout=0.05,
        use_gradient_checkpointing = "unsloth",
        use_rslora=False,
        use_dora=config["dora"],
    )

    training_args = SFTConfig(
        output_dir=str(output_root / f"{config['name']}_runs"),
        save_strategy="steps",
        save_steps=10,
        save_total_limit=10,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation,
        weight_decay = 0.001,
        learning_rate=learning_rate,
        logging_steps=5,
    )

    trainer = SFTTrainer(
        model=model,
        processing_class=tokenizer,
        train_dataset=train_ds,
        args=training_args,
    )
    
    if TRAIN_CONVERSATION and RESPONSES_ONLY:
        trainer = train_on_responses_only(
            trainer,
            instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
            response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
        )
    

    print(f"Training on {len(train_ds)} examples...")
    stats = trainer.train()

    adapter_dir = output_root / f"{config['name']}_adapter"
    adapter_dir.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(adapter_dir)
    tokenizer.save_pretrained(adapter_dir)

    # Cleanup VRAM for next run
    del trainer
    # del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print(f"Saved {config['name']} adapter to {adapter_dir}")


In [5]:
# Cell 5: Run sequential A/B training
for cfg in configs:
    pass
train_adapter(configs[0]) # Train LoRA only

# print("\n" + "="*60)
# print("✨ Training complete!")
# print("="*60)
# print(f"\nAdapters saved to: {output_root}")
# print(f"  - {output_root / 'lora_adapter'}")
# print(f"  - {output_root / 'dora_adapter'}")


🚀 Training LORA adapter...
==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 4070 Ti. Num GPUs = 1. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.10.0+cu130. CUDA: 8.9. CUDA Toolkit: 13.0. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/31885 [00:00<?, ? examples/s]

Unsloth 2026.2.1 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


Unsloth: Tokenizing ["text"]:   0%|          | 0/31885 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.
Training on 31885 examples...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 31,885 | Num Epochs = 1 | Total steps = 3,986
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 114,032,640 of 12,361,815,040 (0.92% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,3.605800
10,3.935900
15,3.685200
20,3.685400
25,3.542200
30,3.130100
35,2.885500
40,2.669400
45,2.374200
50,2.176900


Saved lora adapter to C:\Users\a\Desktop\other SHIET\fine-poems\outputs\lora_adapter


In [6]:
from transformers import TextStreamer
del model
# Test to see if it works
def load_adapter(adapter_dir: Path):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_id,
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=True,
    )
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = 'llama-3.1',
    )
    model.load_adapter(adapter_dir)
    FastLanguageModel.for_inference(model)
    return model, tokenizer

# Cell 4: Inference helper
def generate_reply(model, tokenizer, prompt: str):
    messages = [
        {'role': 'system', 'content': "You are a helpful AI assistant."},
        {'role': 'user', 'content': prompt},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors = "pt",
        tokenize = True,
        add_generation_prompt = True,
    ).to('cuda')
    return model.generate(input_ids=inputs, max_new_tokens = max_seq_length, use_cache = True, temperature = 1.5, streamer = TextStreamer(tokenizer))

prompt="Should I go for my dreams and quit my cushy job or keep at it but not be as invested in it?"
model, tokenizer = load_adapter(output_root / "lora_adapter")
generate_reply(model, tokenizer, prompt)

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 4070 Ti. Num GPUs = 1. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.10.0+cu130. CUDA: 8.9. CUDA Toolkit: 13.0. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 